In [8]:
!bash --login -c "poetry update"

Updating dependencies
Resolving dependencies... (9.0s)

No dependencies to install or update


# Option valuation model based on historical prices

This code assumes `aws configure` has been run in the environment of the notebook.

In [1]:
import optionspricing
import boto3
import helpers
import pandas
from dotenv import load_dotenv

load_dotenv()

BINANCE_SYMBOL = 'ETHUSDT'

s3 = boto3.resource('s3')

data_file = helpers.fetch_object(s3, "test-binance-prices-255120844515", f"{BINANCE_SYMBOL}-full.csv.zip")
prices_df = pandas.read_csv(data_file, compression='zip', header=0, index_col="dateTime")
prices_df.index = pandas.to_datetime(prices_df.index)

## Option valuation model: input parameters here

In [2]:
target_period_hours = 3 * 24
strikes_universe_size = 4
CUT_OFF_YEAR_MONTH = (2020, 1)

instrument_code = BINANCE_SYMBOL[:3]

headers = {"Content-Type": "application/json"}
base_url = "https://www.deribit.com/api/v2/public"

trading_model = optionspricing.TradingModel(base_url, headers, instrument_code, target_period_hours)
trading_model.cutoff_year_month(CUT_OFF_YEAR_MONTH)
target_expiry = trading_model.target_expiry
underlying_price = trading_model.underlying_price
remaining_hours = trading_model.remaining_hours

option_chain_df = trading_model.evaluate(prices_df, strikes_universe_size)
simulation = trading_model.simulate_strategy_long_straddle(option_chain_df, strikes_universe_size)
print(simulation)
simulation.allocate([1, 2], 1500. * 2./100.)

target expiry: Sun 10 Mar, 08:00 (63 hours left)
current price: 3864.93
-------------------------------
trading put 3850 and call 3900 (hit ratio: 39%)
cost: 0.041 / value: 0.045, benefit/cost = 1.1x
($) cost: 160.39 / value: 172.48, average gain = 12.08
-------------------------------
trading put 3825 and call 3925 (hit ratio: 38%)
cost: 0.036 / value: 0.039, benefit/cost = 1.1x
($) cost: 137.21 / value: 151.77, average gain = 14.56
-------------------------------
trading put 3800 and call 3950 (hit ratio: 37%)
cost: 0.031 / value: 0.035, benefit/cost = 1.1x
($) cost: 119.81 / value: 133.49, average gain = 13.67
-------------------------------
trading put 3775 and call 3975 (hit ratio: 35%)
cost: 0.027 / value: 0.030, benefit/cost = 1.1x
($) cost: 104.35 / value: 117.37, average gain = 13.01


[{'put': 3825.0, 'call': 3925.0, 'quantity': 0},
 {'put': 3800.0, 'call': 3950.0, 'quantity': 0}]

In [61]:
simulation.options_chain

,value_call,value_call_pct,call_bid,call_ask,value_put,value_put_pct,put_bid,put_ask
strike,,,,,,,,
3675.0,165.970299,NaN,0.0385,0.0445,36.721278,0.009715,0.0115,0.0120
3700.0,147.100489,NaN,0.0340,0.0400,42.851468,0.011337,0.0135,0.0145
3725.0,129.417127,NaN,0.0315,0.0340,50.168105,0.013273,0.0160,0.0170
3750.0,113.098046,NaN,0.0280,0.0305,58.849025,0.015569,0.0190,0.0200
3775.0,98.406021,NaN,0.0255,0.0265,69.157000,0.018297,0.0220,0.0235
3800.0,85.527160,0.022627,0.0225,0.0235,81.278139,NaN,0.0255,0.0270
3825.0,74.329739,0.019665,0.0195,0.0210,95.080717,NaN,0.0285,0.0315
3850.0,64.558905,0.017080,0.0170,0.0180,110.309884,NaN,0.0325,0.0355
3875.0,55.989082,0.014813,0.0150,0.0160,126.740061,NaN,0.0370,0.0405


# Sanity checks

In [3]:
trading_model.valuation.columns

Index(['prices', 'period_close_series', 'strike_1', 'strike_pct_1',
       'call_value_pct_1', 'put_value_pct_1', 'strike_2', 'strike_pct_2',
       'call_value_pct_2', 'put_value_pct_2', 'strike_3', 'strike_pct_3',
       'call_value_pct_3', 'put_value_pct_3', 'strike_4', 'strike_pct_4',
       'call_value_pct_4', 'put_value_pct_4', 'strike_5', 'strike_pct_5',
       'call_value_pct_5', 'put_value_pct_5', 'strike_6', 'strike_pct_6',
       'call_value_pct_6', 'put_value_pct_6', 'strike_7', 'strike_pct_7',
       'call_value_pct_7', 'put_value_pct_7', 'strike_8', 'strike_pct_8',
       'call_value_pct_8', 'put_value_pct_8', 'strike_9', 'strike_pct_9',
       'call_value_pct_9', 'put_value_pct_9'],
      dtype='object')